In [37]:
import sys
sys.path.append('../')
import os
import re
import json
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import numpy as np
from logparser import Spell, Drain

In [38]:
base_dir = os.path.expanduser('~/logbert_OPENSTACK/output')
print(base_dir)
nput_dir  = os.path.expanduser('~/logbert_OPENSTACK/input')
output_dir = os.path.expanduser('~/logbert_OPENSTACK/output')

/root/logbert_OPENSTACK/output


In [39]:
def generate_train_test(openstack_sequence_file, n=None, ratio=0.3):
    instanceId_label_dict = {}
    instanceId_label_file = os.path.join(output_dir, "unique_label_filtered.csv")
    instanceId_df = pd.read_csv(instanceId_label_file)
    for _, row in tqdm(instanceId_df.iterrows()):
        instanceId_label_dict[row["instance_id"]] = 1 if row["Labels"] == "ANOMALY" else 0

    seq = pd.read_csv(openstack_sequence_file)
    seq["Labels"] = seq["instance_id"].apply(lambda x: instanceId_label_dict.get(x))
    normal_seq = seq[seq["Labels"] == 0]["EventSequence"]
    normal_seq = normal_seq.sample(frac=1, random_state=20)
    abnormal_seq = seq[seq["Labels"] == 1]["EventSequence"]
    normal_len, abnormal_len = len(normal_seq), len(abnormal_seq)
    train_len = n if n else int(normal_len * ratio)
    print("normal size {0}, abnormal size {1}, training size {2}".format(normal_len, abnormal_len, train_len))

    train = normal_seq.iloc[:train_len]
    test_normal = normal_seq.iloc[train_len:]
    test_abnormal = abnormal_seq

    df_to_file(train, os.path.join(output_dir, "train.txt"))
    df_to_file(test_normal, os.path.join(output_dir, "test_normal.txt"))
    df_to_file(test_abnormal, os.path.join(output_dir,"test_abnormal.txt"))
    print("generate train test data done")


def df_to_file(df, file_name):
    with open(file_name, 'w') as f:
        for row in df:
            f.write(' '.join([str(ele) for ele in eval(row)]))
            f.write('\n')


In [40]:
if __name__ == "__main__":
    log_sequence_file = os.path.join(base_dir , "openstack_sequence.csv")
    generate_train_test(log_sequence_file, n=2050)

2070it [00:00, 22843.29it/s]


normal size 2065, abnormal size 4, training size 2050
generate train test data done
